In [14]:
# Re-run the gif creation process due to earlier interruption
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio.v2 as imageio
import io

# Re-load the CSV file
fulldatasetpath = '/Volumes/Data_Drive/datasets/VIDIMU'
file_path = os.path.join(fulldatasetpath,'dataset','videoandimusyncrop/S40/S40_A01_T01.csv')
df = pd.read_csv(file_path)
df = df.iloc[:,1:]
print(df.columns[:3])
joint_names_updated = [df.columns[j][:-2] for j in range(1, 103, 3)]

Index(['pelvis_x', 'pelvis_y', 'pelvis_z'], dtype='object')


In [15]:
print(joint_names_updated)

['pelvis', 'left_hip', 'right_hip', 'torso', 'left_knee', 'right_knee', 'neck', 'left_ankle', 'right_ankle', 'left_big_toe', 'right_big_toe', 'left_small_toe', 'right_small_toe', 'left_heel', 'right_heel', 'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky_knuckle', 'right_pinky_knuckle', 'left_middle_tip', 'right_middle_tip', 'left_index_knuckle', 'right_index_knuckle', 'left_thumb_tip', ' right_thumb_tip']


## Normalize

In [16]:
# # Center the skeleton at the pelvis
# df_centered = df.copy()
# df_centered[[f"{joint}_x" for joint in joint_names_updated]] -= df["pelvis_x"].values[:, None]
# df_centered[[f"{joint}_y" for joint in joint_names_updated]] -= df["pelvis_y"].values[:, None]
# df_centered[[f"{joint}_z" for joint in joint_names_updated]] -= df["pelvis_z"].values[:, None]

# # Small epsilon to avoid division by zero
# epsilon = 1e-6

# # Compute the left-to-right hip vector and normalize it
# left_to_right_hip = df_centered[["right_hip_x", "right_hip_y", "right_hip_z"]].values - \
#                     df_centered[["left_hip_x", "left_hip_y", "left_hip_z"]].values
# left_to_right_hip /= (np.linalg.norm(left_to_right_hip, axis=1)[:, None] + epsilon)

# # Compute the pelvis-to-torso vector and normalize it
# pelvis_to_torso = df_centered[["torso_x", "torso_y", "torso_z"]].values
# pelvis_to_torso /= (np.linalg.norm(pelvis_to_torso, axis=1)[:, None] + epsilon)

# # Calculate the front-facing direction using the cross product
# front_facing = np.cross(left_to_right_hip, pelvis_to_torso)
# front_facing /= (np.linalg.norm(front_facing, axis=1)[:, None] + epsilon)

# # Recalculate the upward direction to ensure orthogonality
# upward_direction = np.cross(front_facing, left_to_right_hip)
# upward_direction /= (np.linalg.norm(upward_direction, axis=1)[:, None] + epsilon)

# # Set up reference vertical axis from the first frame
# reference_vertical_axis = pelvis_to_torso[0]  # Use the first frame as the reference
# reference_vertical_axis /= np.linalg.norm(reference_vertical_axis)  # Normalize

# # Calculate the rotation needed for each frame to align the vertical axis with the reference
# rotation_matrices = []
# for i in range(pelvis_to_torso.shape[0]):
#     # Current vertical axis for this frame
#     current_vertical_axis = pelvis_to_torso[i]
    
#     # Compute the rotation axis (cross product of reference and current vertical axis)
#     rotation_axis = np.cross(reference_vertical_axis, current_vertical_axis)
#     rotation_axis_norm = np.linalg.norm(rotation_axis)
    
#     if rotation_axis_norm < epsilon:
#         # If the axis norm is too small, skip rotation (axes are nearly aligned)
#         rotation_matrix = np.eye(3)
#     else:
#         rotation_axis /= rotation_axis_norm  # Normalize the rotation axis
        
#         # Compute the angle between the current and reference vertical axes
#         angle = np.arccos(np.clip(np.dot(reference_vertical_axis, current_vertical_axis), -1.0, 1.0))
        
#         # Use Rodrigues' rotation formula to compute the rotation matrix
#         K = np.array([
#             [0, -rotation_axis[2], rotation_axis[1]],
#             [rotation_axis[2], 0, -rotation_axis[0]],
#             [-rotation_axis[1], rotation_axis[0], 0]
#         ])
#         I = np.eye(3)
#         rotation_matrix = I + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)
    
#     # Append the rotation matrix for this frame
#     rotation_matrices.append(rotation_matrix)

# rotation_matrices = np.array(rotation_matrices)  # Convert to a NumPy array

# # Apply the calculated rotation matrices to align the joint coordinates
# aligned_df = df_centered.copy()
# for joint in joint_names_updated:
#     # Extract the 3D coordinates for the joint
#     joint_coords = df_centered[[f"{joint}_x", f"{joint}_y", f"{joint}_z"]].values
    
#     # Apply the rotation matrix to align the joint coordinates
#     aligned_coords = np.einsum('ijk,ik->ij', rotation_matrices, joint_coords)
    
#     # Update the aligned DataFrame with the new coordinates
#     aligned_df[f"{joint}_x"], aligned_df[f"{joint}_y"], aligned_df[f"{joint}_z"] = aligned_coords.T


In [17]:
import numpy as np

def align_torso_above_pelvis(df, joint_names, vertical_axis=np.array([0.0, 1.0, 0.0]), epsilon=1e-6):
    """
    Aligns the torso vertically above the pelvis for each frame in the dataset.
    
    Parameters:
    - df: DataFrame containing the original 3D coordinates of joints
    - joint_names: List of joint names to transform
    - vertical_axis: The desired global vertical axis for standardization (default: [0, 1, 0])
    - epsilon: Small value to avoid division by zero
    
    Returns:
    - aligned_df: DataFrame containing the aligned coordinates
    """
    
    # Convert vertical_axis to float to avoid casting issues
    vertical_axis = np.asarray(vertical_axis, dtype=np.float64)
    vertical_axis /= np.linalg.norm(vertical_axis)  # Normalize the vertical axis
    
    # Center the skeleton at the pelvis
    df_centered = df.copy()
    df_centered[[f"{joint}_x" for joint in joint_names]] -= df["pelvis_x"].values[:, None]
    df_centered[[f"{joint}_y" for joint in joint_names]] -= df["pelvis_y"].values[:, None]
    df_centered[[f"{joint}_z" for joint in joint_names]] -= df["pelvis_z"].values[:, None]

    # Compute the pelvis-to-torso vector and normalize it
    pelvis_to_torso = df_centered[["torso_x", "torso_y", "torso_z"]].values
    pelvis_to_torso /= (np.linalg.norm(pelvis_to_torso, axis=1)[:, None] + epsilon)

    # Calculate the rotation needed to align the pelvis-to-torso with the vertical axis
    rotation_matrices = []
    for i in range(pelvis_to_torso.shape[0]):
        current_vertical_axis = pelvis_to_torso[i]
        
        # Compute the rotation axis (cross product of vertical axis and current vertical axis)
        rotation_axis = np.cross(current_vertical_axis, vertical_axis)
        rotation_axis_norm = np.linalg.norm(rotation_axis)
        
        if rotation_axis_norm < epsilon:
            # If the axis norm is too small, skip rotation (axes are nearly aligned)
            rotation_matrix = np.eye(3)
        else:
            rotation_axis /= rotation_axis_norm  # Normalize the rotation axis
            
            # Compute the angle between the current and vertical axes
            angle = np.arccos(np.clip(np.dot(current_vertical_axis, vertical_axis), -1.0, 1.0))
            
            # Use Rodrigues' rotation formula to compute the rotation matrix
            K = np.array([
                [0, -rotation_axis[2], rotation_axis[1]],
                [rotation_axis[2], 0, -rotation_axis[0]],
                [-rotation_axis[1], rotation_axis[0], 0]
            ])
            I = np.eye(3)
            rotation_matrix = I + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)
        
        # Append the rotation matrix for this frame
        rotation_matrices.append(rotation_matrix)

    rotation_matrices = np.array(rotation_matrices)  # Convert to a NumPy array

    # Apply the rotation matrices to align the joint coordinates
    aligned_df = df_centered.copy()
    for joint in joint_names:
        joint_coords = df_centered[[f"{joint}_x", f"{joint}_y", f"{joint}_z"]].values
        aligned_coords = np.einsum('ijk,ik->ij', rotation_matrices, joint_coords)
        aligned_df[f"{joint}_x"], aligned_df[f"{joint}_y"], aligned_df[f"{joint}_z"] = aligned_coords.T
    
    return aligned_df

# Example usage:
aligned_df = align_torso_above_pelvis(df, joint_names_updated, vertical_axis=np.array([0, 1, 0]))


## Plot

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import io
import imageio.v2 as imageio

# Extract the XYZ coordinates for each joint
joint_coords_updated = {}
for joint in joint_names_updated:
    joint_coords_updated[joint] = aligned_df[[f'{joint}_x', f'{joint}_y', f'{joint}_z']].values

# Get the min and max values for each axis (for consistent plot scaling)
x_min, x_max = aligned_df[[f'{joint}_x' for joint in joint_names_updated]].min().min(), aligned_df[[f'{joint}_x' for joint in joint_names_updated]].max().max()
y_min, y_max = aligned_df[[f'{joint}_y' for joint in joint_names_updated]].min().min(), aligned_df[[f'{joint}_y' for joint in joint_names_updated]].max().max()
z_min, z_max = aligned_df[[f'{joint}_z' for joint in joint_names_updated]].min().min(), aligned_df[[f'{joint}_z' for joint in joint_names_updated]].max().max()

# Function to plot the skeleton with a fixed view and grid
def plot_skeleton_frame_with_axes(ax, frame_idx):
    joints = {joint: joint_coords_updated[joint][frame_idx] for joint in joint_names_updated}

    # Updated skeleton connections based on the available joints
    connections_updated = [
        ('pelvis', 'left_hip'), ('pelvis', 'right_hip'),
        ('left_hip', 'left_knee'), ('right_hip', 'right_knee'),
        ('left_knee', 'left_ankle'), ('right_knee', 'right_ankle'),
        ('left_ankle', 'left_heel'), ('right_ankle', 'right_heel'),
        ('left_heel', 'left_big_toe'), ('right_heel', 'right_big_toe'),
        ('left_heel', 'left_small_toe'), ('right_heel', 'right_small_toe'),
        ('pelvis', 'torso'), ('torso', 'neck'), ('neck', 'nose'),
        ('neck', 'left_shoulder'), ('neck', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'), ('right_shoulder', 'right_elbow'),
        ('left_elbow', 'left_wrist'), ('right_elbow', 'right_wrist'),
        ('left_wrist', 'left_pinky_knuckle'), ('right_wrist', 'right_pinky_knuckle'),
        ('left_wrist', 'left_index_knuckle'), ('right_wrist', 'right_index_knuckle'),
        ('left_wrist', 'left_thumb_tip'), ('right_wrist', 'right_thumb_tip')
    ]

    # Extract joint positions for plotting
    xs, ys, zs = [], [], []
    for joint in joint_names_updated:
        if joint in joints:  # Ensure the joint is available in the current frame
            x, y, z = joints[joint]
            xs.append(x)
            ys.append(y)
            zs.append(z)

    # Plot joints and connections
    ax.scatter(xs, ys, zs, color='blue')
    for conn in connections_updated:
        if conn[0] in joints and conn[1] in joints:  # Ensure both joints in the connection exist
            x_vals = [joints[conn[0]][0], joints[conn[1]][0]]
            y_vals = [joints[conn[0]][1], joints[conn[1]][1]]
            z_vals = [joints[conn[0]][2], joints[conn[1]][2]]
            ax.plot(x_vals, y_vals, z_vals, color='red')

    # Set the axis limits dynamically based on the entire dataset
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_zlim([z_min, z_max])

    # Show grid and axes
    ax.grid(True)
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')

# Create gif with dynamic movement and fixed view
gif_images_dynamic = []
for frame in range(0, aligned_df.shape[0], 5):  # Every 10th frame for speed
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Set a predefined view
    ax.view_init(elev=30, azim=-90)  # Change these values to get the desired initial perspective

    # Plot the skeleton for this frame
    plot_skeleton_frame_with_axes(ax, frame)

    # Save figure to a buffer instead of a file
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    gif_images_dynamic.append(imageio.imread(buf))
    buf.close()
    plt.close(fig)  # Close the figure after saving to buffer

# Save the gif without temporary files
imageio.mimsave('skeleton_3d_with_axes_dynamic.gif', gif_images_dynamic, fps=10)